<a href="https://www.kaggle.com/code/fakhrizalahadiat/eda-telco-customer-churn?scriptVersionId=143046340" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

#Dataset Information

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

Terdapat 1 kolom yang tipe datanya seharusnya float yaitu kolom `TotalCharges` sehingga harus dirubah terlebih dahulu

#Missing Value Check

In [ ]:
#mengecek data total charges yang mengandung string
df.loc[df["TotalCharges"].str.contains(" ")]

Terdapat 11 data `TotalCharges` yang mengandung string sehingga data tersebut tidak akan masukkan untuk analisis

In [ ]:
#mengambil data TotalCharges yang tidak mengandung string
df = df.loc[~df["TotalCharges"].str.contains(" ")]
#merubah tipe data pada kolom TotalCharges menjadi float
df["TotalCharges"] = df["TotalCharges"].astype(float)

In [ ]:
df.info()

Kolom `TotalCharges` sudah berubah tipe datanya menjadi float dan total data menjadi 7032





In [ ]:
df.isna().sum()

data sudah tidak terdapat Missing Value

# Duplicated Check

In [ ]:
df.duplicated().sum()

tidak terdapat data yang duplikat

### Observation

Pada kolom `TotalCharges` sebelumnya tipe data object dirubah menjadi float terdapat error karena mengandung string. Setelah dicek ternyata terdapat data yang mengandung string `whitespace` sehingga diambil hanya data yang tidak mengandung string sehingga jumlah data menjadi 7032, setelah itu dilakukan konversi ke tipe data float

# Data Understanding

## Statistical Summary

In [ ]:
#memisahkan kolom kategorikal dan numeric
categorical = ["customerID","gender","Partner","Dependents","PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies","Contract","PaperlessBilling","PaymentMethod","Churn"]
numerical = ["SeniorCitizen","tenure","MonthlyCharges","TotalCharges"]

In [ ]:
#numerical statistical summary
df[numerical].describe()

Terlihat bahwa terdapat jenis data discrete dan continous pada data numerik

1.   Discrete : SeniorCitizen
2.   Continous : tenure, MonthlyCharges, TotalCharges



In [ ]:
#melihat banyak value pada kolom discrete
df["SeniorCitizen"].value_counts()

**Observation:**
1. Untuk masing-masing kolom min dan max value tidak terdapat masalah
2. `SeniorCitizen` merupakan tipe data boolean karena hanya terdapat 2 unique Value 0 atau 1
3. `tenure` Mean lebih besar dibandingkan median sehingga distribusi cenderung positif skew
4. `MonthlyCharges` Mean lebih kecil dibandingkan median sehingga distribusi cenderung negatif skew
5. `TotalCharges` Mean lebih besar dibandingkan median sehingga distribusi cenderung positif skew

In [ ]:
#categorical statistical summary
df[categorical].describe().T

In [ ]:
#melihat banyak value pada kolom categorical
for col in categorical:
  print(f"Value pada {col} adalah")
  print(df[col].value_counts(), "\n")

**Observation**
1. `gender` memiliki 2 unique value yaitu Male, Female
2. `Partner, Dependents, PhoneService, PaperlessBilling , Churn` merupakan tipe data boolean karena hanya memiliki 2 unique value yaitu Yes atau No
3. `MultipleLines, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, Techsupport, StreamingTV, StreamingMovies, Contract` hanya memiliki tipe 3 data unique
4. `PaymentMethod` memiliki 4 unique value
5. Terlihat bahwa customer yang paling banyak yaitu pada gender male dan tidak memiliki partner dan dependent serta mayoritas customer memiliki phone service dengan internet paling banyak menggunakan FiberOptic
6. Mayoritas customer tidak memiliki multipleline, online security, online backup, device protection, tech support, streaming TV dan streaming Movies
7. Mayoritas contract pembayarannya Month to Month, billingnya paperless dan metode pembayarannya Electronic Check

# Univariate Analysis

### Boxplot untuk melihat adanya outliers

In [ ]:
plt.figure(figsize = (12,4))

features = numerical
for i in range(0,len(features)):
  plt.subplot(1,len(features), i+1)
  sns.boxplot(data = df, y = features[i])
  plt.tight_layout()

In [ ]:
#cek outlier secara mendetail
outlier = df["SeniorCitizen"].max()

df[df["SeniorCitizen"] == outlier]

**Observation**

Terdapat 1142 outlier pada kolom `SeniorCitizen` dengan value = 1 tetapi hal tersebut tidak menjadi masalah karena tipe data kolom `SeniorCitizen` yaitu boolean dan hanya terdapat 2 value yaitu 1 atau 0

In [ ]:
df.info()

### Countplot Churn (Target)

In [ ]:
plt.figure(figsize = (8,4))
ax = sns.countplot(x = df["Churn"])
ax.bar_label(ax.containers[0], color = "black")
plt.title("Distribution Churn")
plt.ylabel("Number of Customers")
plt.show()

**Observation**

Pada barplot kolom `Churn` terlihat adanya ketidakseimbangan distribusi antara yang churnnya Yes atau No sehingga perlu dilakukan analisis lebih lanjut

### Countplot berdasarkan Demografi customer

In [ ]:
features = ["gender","Partner","Dependents","SeniorCitizen"]
plt.figure(figsize = (14,4))
for i in range(0,len(features)):
  plt.subplot(1, len(features), i+1)
  ax = sns.countplot(x=df[features[i]])
  ax.bar_label(ax.containers[0], color = "Black")
  plt.title("Distribution " + features[i])
  plt.ylabel("Number of Customers")
  plt.tight_layout()

**Observation**

1. Terlihat customer berdasarkan `gender` dan `partner` cenderung seimbang
2. Terdapat perbedaan yang signifikan pada kolom `SeniorCitizen` bahwa untuk customer dengan kriteria Senior Citizen hanya sebanyak 16,24% sedangkan 83,75% customer berusia muda
2. Terlihat pada kolom `Dependents` bahwa sebanyak 70% mayoritas customer tidak memiliki tanggungan hal ini sesuai dengan kolom `SeniorCitizen` karena mayoritas customer berusia muda

### Countplot berdasarkan jenis service

In [ ]:
features = ["PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
plt.figure(figsize = (12,16))
for i in range(0,len(features)):
  plt.subplot(4, len(features)//3, i+1)
  ax = sns.countplot(x=df[features[i]])
  ax.bar_label(ax.containers[0], color = "Black")
  plt.title("Distribution " + features[i])
  plt.ylabel("Number of Customers")
  plt.tight_layout()

**Observation**
1. Terlihat customer cenderung memiliki PhoneService dan mayoritas tidak memiliki Multiplelines
2. Untuk internet services mayoritas customer cenderung menggunakan Fiber Optic
3. Untuk kolom OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport masing-masing distribusinya tidak seimbang dan mayoritas customer tidak memiliki layanan tersebut
4. Untuk kolom StreamingTV dan Streaming Movies cenderung seimbang untuk yang punya dan tidak punya layanan tersebut sedangkan ketidakseimbangan terjadi pada customer yang tidak memiliki internet service

### Countplot berdasarkan jenis pembayaran

In [ ]:
features = ["Contract","PaperlessBilling","PaymentMethod",]
plt.figure(figsize = (12,6))
for i in range(0,len(features)):
  plt.subplot(1, len(features), i+1)
  ax = sns.countplot(x=df[features[i]])
  ax.bar_label(ax.containers[0], color = "Black")
  plt.title("Distribution " + features[i])
  plt.ylabel("Number of Customers")
  plt.xticks(rotation = 25)
  plt.tight_layout()

**Observation**
1. Untuk distribusi Contract pembayaran tidak seimbang dan mayoritas customer cenderung melakukan pembayaran yang dilakukan perbulan
2. Mayoritas customer cenderung menggunakan tagihan paperless
3. Pada PaymentMethod terlihat mayoritas customer melakukan pembayaran menggunakan metode pembayaran Elektronik sedangkan metode pembayaran lain cenderung sama

### Histogram dan KDE Plot untuk mengetahui distribusi pada data numerical

In [ ]:
features = numerical

In [ ]:
#membuat histogram untuk melihat jumlah distribusi data
plt.figure(figsize = (12,6))
for i in range(0,len(features)):
  plt.subplot(2, len(features)//2, i+1)
  sns.histplot(x=df[features[i]])
  plt.title("Distribution : " + features[i])
  plt.ylabel("Number of Customers")
  plt.tight_layout()

In [ ]:
#membuat KDE Plot untuk melihat distribusi data numerical
plt.figure(figsize = (12,6))
for i in range(0,len(features)):
  plt.subplot(2, len(features)//2, i+1)
  sns.kdeplot(x=df[features[i]])
  plt.title("Distribution : " + features[i])
  plt.tight_layout()

**Observation**
1. Untuk kolom `SeniorCitizen` merupakan data discrete dan hanya memiliki 2 unique value, terlihat bahwa distribusi paling banyak pada angka 0 dan distribusinya tidak seimbang yang artinya mayoritas customer berusia muda
2. Untuk kolom `tenure` terlihat frekuensi distribusi paling banyak pada sekitar angka 1 - 5 serta distribusi tidak normal dan distribusinya bimodal
3. Untuk kolom `MonthlyCharges` frekuensi distribusi paling banyak pada sekitar angka 20 dan distribusinya tidak normal
4. untuk kolom `TotalCharges` frekuensi distribusi paling banyak diantara angka 0 - 200 dan distribusinya tidak normal/simetris cenderung skew ke kanan

# Bivariate Analysis

### Barplot Churn berdasarkan Customer Demografi

In [ ]:
features = ["gender","Partner","Dependents","SeniorCitizen"]
plt.figure(figsize = (10,8))
for i in range(0,len(features)):
  churn_demografi = df.groupby(features[i],as_index = False)["Churn"].value_counts()
  plt.subplot(2, len(features)//2, i+1)
  ax = sns.barplot(x=features[i], y = "count", hue="Churn", data = churn_demografi)
  ax.bar_label(ax.containers[0], color = "Black")
  ax.bar_label(ax.containers[1], color = "Black")
  plt.title("Churn by " + features[i])
  plt.ylabel("Number of Customers")
  plt.tight_layout()

**Observation**

1. Terlihat bahwa churn tidak berpengaruh dari gender karena jumlahnya cenderung sama
2. Terlihat churn lebih banyak pada customer yang tidak memiliki pasangan
3. Terlihat churn lebih banyak pada customer yang tidak memiliki tanggungan
4. Terlihat churn lebih banyak pada customer yang masih muda tapi untuk persentase churn lebih tinggi pada customer senior dibandingkan customer yang masih muda yaitu sebesar 41,68 %

### Barplot Churn berdasarkan Jenis Service

In [ ]:
features = ["PhoneService","MultipleLines","InternetService","OnlineSecurity","OnlineBackup","DeviceProtection","TechSupport","StreamingTV","StreamingMovies"]
plt.figure(figsize = (14,16))
for i in range(0,len(features)):
  churn_service = df.groupby(features[i],as_index = False)["Churn"].value_counts()
  plt.subplot(4, len(features)//3, i+1)
  ax = sns.barplot(x=features[i], y = "count", hue="Churn", data = churn_service)
  ax.bar_label(ax.containers[0], color = "Black")
  ax.bar_label(ax.containers[1], color = "Black")
  plt.title("Churn by " + features[i])
  plt.ylabel("Number of Customers")
  plt.tight_layout()

**Observation**
1. Untuk kolom phoneservice disini tidak diperlukan karena sudah dicover oleh kolom multiplelines
2. Untuk churn pada multiplelines cenderung sama untuk customer yang memiliki multipleline maupun tidak
3. Untuk churn pada internet service didominasi oleh customer yang memakai internet Fiber Optic dibandingkan dengan jenis internet lain
4. Bahwa terlihat customer churn cenderung banyak kepada customer yang tidak memiliki/diberikan layanan Online Security, OnlineBackup, Device Protection dan Tech Support
5. sedangkan customer churn berdasarkan layanan streamingTV dan streamingMovies cenderung hampir sama sehingga customer churn kemungkinan besar tidak berdasarkan layanan ini

### Barplot Churn berdasarkan Jenis Pembayaran

In [ ]:
features = ["Contract","PaperlessBilling","PaymentMethod",]
plt.figure(figsize = (12,6))
for i in range(0,len(features)):
  churn_jenispembayaran = df.groupby(features[i],as_index = False)["Churn"].value_counts()
  plt.subplot(1, len(features), i+1)
  ax = sns.barplot(x=features[i], y = "count", hue="Churn", data = churn_jenispembayaran)
  ax.bar_label(ax.containers[0], color = "Black")
  ax.bar_label(ax.containers[1], color = "Black")
  plt.title("Distribution " + features[i])
  plt.ylabel("Number of Customers")
  plt.xticks(rotation = 25)
  plt.tight_layout()

**Observation**
1. terlihat bahwa customer churn dominan pada pembayaran yang dilakukan setiap bulan dibandingkan pembayaran lain
2. Customer cenderung churn kepada customer yang biasa melakukan pembayaran melalui pembayaran elektronik dengan tagihan paperless

# Multivariate Analysis

### Heatmap untuk melihat Korelasi antar kolom Numerical

In [ ]:
plt.figure(figsize = (12,6))
corr = df[numerical].corr()
sns.heatmap(corr, annot=True, fmt =".2f", cmap="magma")
plt.show()

**Observation**

Terlihat bahwa ada relasi yang cukup kuat pada tenure dengan TotalCharges dan MonthlyCharges dengan TotalCharges

### Scatterplot untuk melihat sebaran customer yang Churn

Scatterplot Churn by Tenure vs TotalCharges

In [ ]:
sns.scatterplot(data = df, x = "tenure", y="TotalCharges", hue ="Churn", style="Churn")
plt.show()

**Observation**

Terlihat bahwa tenure yang lebih lama dengan total charges yang lebih besar potensi churn lebih tinggi

Scatterplot Churn by Tenure vs MonthlyCharges

In [ ]:
sns.scatterplot(data = df, x = "tenure", y="MonthlyCharges", hue ="Churn", style="Churn")
plt.show()

**Observation**

Terlihat bahwa Churn banyak terjadi pada tenure 0 - 20 bulan dengan MonthlyCharges yang cukup besar

Scatterplot Churn by MonthlyCharges vs TotalCharges

In [ ]:
sns.scatterplot(data = df, x = "MonthlyCharges", y="TotalCharges", hue ="Churn", style="Churn")
plt.show()

**Observation**

Terlihat bahwa adanya hubungan yang kuat dimana semakin besar Monthlycharges semakin besar juga TotalCharges, disini dapat dilihat juga terjadinya Churn lebih banyak pada MonthlyCharges yang lebih tinggi dengan TotalCharges yang lebih rendah

### Pairplot untuk melihat customer churn mendetail pada data tipe numerik

In [ ]:
sns.pairplot(df, hue="Churn")
plt.tight_layout()
plt.show()

**Observation**

Bahwa terlihat Churn terjadi pada tenure 0 - 20 bulan dengan MonthlyCharges yang besar pada tenure tersebut dan terlihat juga jumlah churn paling banyak pada customer berusia muda tetapi kalau dilihat berdasarkan persentase Churn justru churn terjadi lebih banyak pada customer yang Senior dibandingkan customer berusia muda

# Deep Dive Exploration

### Berapa Rata-rata MonthlyCharges dan TotalCharges untuk customer yang mengalami churn ?

In [ ]:
condition = df["Churn"] == "Yes"
customer_churn = df[condition].groupby(["SeniorCitizen"]).agg(mean_monthlycharges = ("MonthlyCharges", "mean"),
                                                    mean_totalcharges = ("TotalCharges","mean")).reset_index()
customer_churn

In [ ]:
plt.figure(figsize = (8,4))

#barplot rata-rata monthlycharges churn
plt.subplot(1,2,1)
ax = sns.barplot(data = customer_churn, x = "SeniorCitizen", y="mean_monthlycharges", palette="deep")
ax.bar_label(ax.containers[0], color ="Black", fmt = "%.2f")
ax.set_title("Mean MonthlyCharges Churn")
ax.set_ylabel("MonthlyCharges")

#barplot rata-rata totalcharges churn
plt.subplot(1,2,2)
ax = sns.barplot(data = customer_churn, x = "SeniorCitizen", y="mean_totalcharges", palette="deep")
ax.bar_label(ax.containers[0], color ="Black", fmt = "%.2f")
ax.set_title("Mean TotalCharges Churn")
ax.set_ylabel("TotalCharges")
plt.tight_layout()
plt.show()

**Observation**

Bahwa terlihat ada perbedaan churn dimana Customer senior terjadi churn dengan rata-rata MonthlyCharges dan TotalCharges yang lebih besar dibandingkan dengan customer yang masih muda. Dapat disimpulkan customer senior churn karena tagihannya yang besar

### Berapa rata-rata customer churn berdasarkan internet services ?



In [ ]:
#membuat feature baru
df['TotalServices'] = (df[['PhoneService', 'InternetService', 'OnlineSecurity',
                                       'OnlineBackup', 'DeviceProtection', 'TechSupport',
                                       'StreamingTV', 'StreamingMovies']]== 'Yes').sum(axis=1)

In [ ]:
condition = df["Churn"] == "Yes"
services_churn = df[condition].groupby(["SeniorCitizen","TotalServices"]).agg(count_totalservices = ("TotalServices","count")).reset_index()
services_churn

In [ ]:
plt.figure(figsize=(8,4))
ax = sns.barplot(data = services_churn, x = "TotalServices", y ="count_totalservices", hue="SeniorCitizen", palette="deep")
ax.bar_label(ax.containers[0], color="Black")
ax.bar_label(ax.containers[1], color="Black")
ax.set_title("Churn by Total Services")
ax.set_ylim(ymax = 450)
plt.show()

**Observation**

Terlihat bahwa churn banyak terjadi pada customer yang hanya memiliki 1 sampai 3 layanan sedangkan churn terjadi sedikit kepada customer yang memiliki semua layanan

### Senior Customer Churn berdasarkan tenure

In [ ]:
condition = (df["Churn"] == "Yes") & (df["SeniorCitizen"] == 1)
tenure_churn = df[condition].groupby(["SeniorCitizen","tenure"]).agg(count = ("tenure","count") ).reset_index().sort_values("count", ascending = False).head(10)
tenure_churn

In [ ]:
plt.figure(figsize=(8,4))
ax = sns.barplot(data = tenure_churn, x = "tenure", y ="count", palette="deep")
ax.bar_label(ax.containers[0], color="Black")
ax.set_title("Highest Churn Senior Customer by Tenure")
ax.set_ylim(ymax = 80)
plt.show()

**Observation**

Bahwa churn yang terjadi pada customer senior lebih banyak pada tenure 1 bulan hal ini kemungkinan terjadi karena MonthlyCharges yang besar dan hanya memiliki layanan yang terbatas

# EDA Conclusion

1. Dataset pada kolom TotalCharges sebelumnya tipe data object dirubah menjadi float terdapat error karena mengandung string. Setelah dicek ternyata terdapat data yang mengandung string whitespace sehingga diambil hanya data yang tidak mengandung string sehingga jumlah data menjadi 7032, setelah itu dilakukan konversi ke tipe data float
2. Tidak terdapat data yang duplikat
3. Untuk kolom SeniorCitizen merupakan data discrete dan hanya memiliki 2 unique value
4. Untuk kolom tenure terlihat frekuensi distribusi paling banyak pada sekitar angka 1 - 5 serta distribusi tidak normal dan distribusinya bimodal
5. Untuk kolom MonthlyCharges frekuensi distribusi paling banyak pada sekitar angka 20 dan distribusinya tidak normal
6. untuk kolom TotalCharges frekuensi distribusi paling banyak diantara angka 0 - 200 dan distribusinya tidak normal/simetris cenderung skew ke kanan
7. Terlihat bahwa ada relasi yang cukup kuat pada tenure dengan TotalCharges dan MonthlyCharges dengan TotalCharges
8. Ditemukan bahwa gender dan partner tidak mempengaruhi adanya churn yang tinggi karena dapat dilihat jumlah churn pada gender dan partner hampir sama
9. Mayoritas Customer berusia muda karena dapat dilihat bahwa mayoritas customer tidak memiliki tanggungan
10. Churn paling banyak pada customer yang berusia muda tetapi berdasarkan persentase, justru customer senior yang memiliki persentase churn paling tinggi
11. Churn banyak terjadi pada tenure 0 - 20 bulan dikarenakan Monthly Charges dan TotalCharges yang tinggi kemungkinan faktor ini salah satu penyebab churn
12. Churn terjadi juga pada customer yang hanya memiliki 1 - 3 layanan, layanan yang minim inilah penyebab terjadinya churn dan mayoritas menggunakan internet Fiber Optic